In [ ]:
import numpy as np

class NeuralNet:
    def __init__(self, layers, epochs, lr, momentum, function, perc_validation):
        self.L = len(layers)    #Number of layers
        self.n = layers.copy()  #An array with the number of units in each layer

        self.h = []             #An array of arrays for the fields (h)
        for lay in range(self.L):
            self.h.append(np.zeros(layers[lay]))
        self.xi = []            #An array of arrays for the activations(Xi)
        for lay in range(self.L):
            self.xi.append(np.zeros(layers[lay]))

        self.w = []             #An array of matrices for the weights
        self.w.append(np.zeros((1, 1)))
        for lay in range(1, self.L):
            self.w.append(np.zeros((layers[lay], layers[lay - 1])))

        self.theta = []         #An array of arrays for thresholds
        for lay in range(self.L):
            self.theta.append(np.zeros(layers[lay]))
        
        self.delta = []         #An array of arrays for the propagation errors
        self.d_w = []           #An array of matrices for the changes on weights
        self.d_theta = []       #An array of arrays for the changes on thresholds
        self.d_w_prev = []      #An array of matrices for the previoius changes of the 
                                #weights used for the momentum term
        self.d_theta_prev = []  #An array of arrays for the previous changes of the 
                                #thresholds used for the momentum term
        self.fact = function      #The name of the activation function that will be 
                                #used (sigmoid, relu, linear, tanh)
        
        self.epochs = epochs
        self.lr = lr
        self.momentum = momentum
        self.perc_validation = perc_validation
                                
        
    
    def initialize_w_theta(self):
        for L in range(1, self.L):
            self.w[L] = np.random.randn(self.n[L], self.n[L-1]) * 0.5
        for L in range(1, self.L):
            self.theta[L] = np.random.randn(self.n[L]) * 0.5
    
    def feed_forward(self, X):
        self.xi[0] = X
        for L in range(1, self.L):
            num_neurons = self.n[L]
            for i in range (0, num_neurons):
                self.xi[L][i] = self.activation_function(i, L)
                
        return self.xi[L]
    
    def activation_function(self, i, L):
        self.h[L][i] = 0
        for j in range(0, self.n[L-1]):
            self.h[L][i] += self.w[L][i, j] * self.xi[L-1][j]
            
        self.h[L][i] -= self.theta[L][i]
            
        return (1 / (1 + np.exp(-self.h[L][i])))
    
    def error_back_propagation(self, out_x, y):
        
        L = self.L
        self.delta[L] = (out_x - y) * self.xi[L] * (1 - self.xi[L])
        
        for L in range(self.L - 1, 0, -1):
            for j in range(0, self.n[L]):
                sum_delta = 0
                for i in range(self.n[L+1]):
                    sum_delta += self.delta[L+1][i] * self.w[L+1][i,j]
                self.delta[L][j] = self.xi[L][j] * (1 - self.xi[L][j]) * sum_delta
        
        
                   
    def fit(self,X,y):
        self.initialize_w_theta()
        
        for epoch in range(self.epochs):
            for pat in range(X.shape[0]):
                #Choose a random pattern (xu zu) of training set
                output = self.feed_forward(X[pat])
                print(output)    
                #Back-propagate the error for this pattern
                #Update the weights and threseholds
            #Feed-forward all training patterns
            #Feed-forward all validation paterns
        #Feed-forward all test partterns
        #Descale and evaluate
        
    def predict(self, X):
        return np.array([self.feed_forward(x) for x in X])
    def loss_epochs():
        return 0


In [20]:
X_train = np.array([
    [0.1, 0.2, 0.3, 0.4],
    [0.5, 0.4, 0.3, 0.2],
    [0.9, 0.8, 0.7, 0.6]
])      

layers = [4, 9, 5, 1]
nn = NeuralNet(layers, 10, 0, 0, "sigmoid", 0)

"""
print("L = ", nn.L, end="\n")
print("n = ", nn.n, end="\n")

print("xi = ", nn.xi, end="\n")
print("xi[0] = ", nn.xi[0], end="\n")
print("xi[1] = ", nn.xi[0], end="\n")

print("wh = ", nn.w, end="\n")
print("wh[1] = ", nn.w[1], end="\n")
"""

# Inicializamos pesos y bias
nn.initialize_w_theta()

# --- Probamos feed-forward para cada patrón ---
print("Feed-forward outputs:")
for idx, x in enumerate(X_train):
    output = nn.feed_forward(x)
    print(f"Input {idx}: {x} -> Output: {output}")
   
print("Activaciones capa 0:", nn.xi[0]) 
print("Activaciones capa 1:", nn.xi[1])
print("Activaciones capa 2:", nn.xi[2])
print("Activaciones capa 3:", nn.xi[3])



Feed-forward outputs:
Input 0: [0.1 0.2 0.3 0.4] -> Output: [0.71914482]
Input 1: [0.5 0.4 0.3 0.2] -> Output: [0.71683582]
Input 2: [0.9 0.8 0.7 0.6] -> Output: [0.7111944]
Activaciones capa 0: [0.9 0.8 0.7 0.6]
Activaciones capa 1: [0.16410673 0.75965658 0.31523687 0.55111987 0.33253429 0.35317588
 0.2784416  0.60745961 0.60892749]
Activaciones capa 2: [0.37138557 0.42852333 0.58074    0.49252724 0.75620211]
Activaciones capa 3: [0.7111944]
